In [ ]:
from typing import Annotated, Literal
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END, START, StateGraph
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_groq import ChatGroq


In [ ]:
load_dotenv()

In [ ]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b", api_key="api_key")

In [ ]:
groq_model.invoke("Hi").content

In [ ]:
def clean_text(text):
    import re
    clean_text = re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL)
    return clean_text

In [ ]:
response = groq_model.invoke("hi")
clean_text(response.content)

In [ ]:
tavily_search = TavilySearchResults()

In [ ]:
tavily_search.invoke("What is GDP?")

In [ ]:
code = """
x = 5
y = x * 2
print(y)
"""

In [ ]:
repl = PythonREPL()

In [ ]:
repl.run(code)

In [ ]:
@tool
def python_repl_text(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """ Use this to execute python code and do math. If you want to see the output of a value.
    you should print it out with 'print(...)'. This is visible to the user."""

    try:
        result = repl.run(code)

    except BaseException as e:
        return f"Failed to execute. Error: {e}"
    
    return f"Successfully executed: {code} Stdout: {result}"

In [ ]:
print(python_repl_text.invoke(code))

In [ ]:
members = ["researcher", "coder"]

In [ ]:
options = members+["FINISH"]

In [ ]:
class Routor(TypedDict):
    """ Worker to route to next. if no workers needed, route to FINISH."""
    next: Literal["researcher", "coder", "FINISH"]

In [ ]:
class State(MessagesState):
    next : str

In [ ]:
system_prompt = f"""
You are a supervisor, tasked with managing a conversation between the following workers: {members}.
Given the following user request, respond with the worker to act next.
Each worker will perform a task and respond with their results and status.
When finished, respond with FINISH.
"""

In [ ]:
[{"role": "system", "content": system_prompt}]

In [ ]:
state = {"next": ["hi"]}

In [ ]:
state["next"]

In [ ]:
[{"role": "system", "content": system_prompt},] + state["next"]

In [ ]:
def supervisor_node(state: State) -> Command[Literal["researcher", "coder", "__end__"]]:
    messages = [{"role": "system", "content": system_prompt},] + state["messages"]
    response = groq_model.with_structured_output(Routor).invoke(messages)
    goto = response("next")
    print("below is my goto***************************")
    print(goto)
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"role": goto})

In [ ]:
def research_node(state: State) -> Command[Literal["supervisor"]]:
    research_agent = create_react_agent(groq_model, tools=[tavily_search], prompt="You are a researcher. DO NOT do any math.")
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor"
    )

In [ ]:
def code_node(state:State) -> Command[Literal["supervisor"]]:
    code_agent = create_react_agent(groq_model, tools=[python_repl_text])
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor"
    )

In [ ]:
# Graph

graph = StateGraph(StateGraph)

graph.add_node("supervisor", supervisor_node)
graph.add_node("researcher", research_node)
graph.add_node("codeer", code_node)

graph.add_edge(START, "supervisor")


app = graph.compile()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
# With the graph created , we can now invoke it and see how it performs:
for s in app.stream({"messages": [("user", "What's the square root of 72?")]}, subgraphs=True):
    print(s)
    print("-----")

In [ ]:
app.invoke({"messages": [("user", "What's the square root of 72?")]})

In [ ]:
app.invoke({"messages": [("user", "What's the afficient python code to get prime nubmers?")]}, subgraphs=True)